In [2]:
import json
import boto3
import requests
from datetime import datetime, timezone
import pandas as pd
from io import StringIO


In [3]:
url = f"https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=-22.9056&longitude=-47.0608&start_date=2024-01-01&end_date=2024-12-31&hourly=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,rain,weather_code,cloud_cover,wind_direction_10m,wind_speed_10m,is_day&timezone=America%2FSao_Paulo"
# Requisição à API
response = requests.get(url)
data = response.json()

In [4]:
# Converter para DataFrame
df = pd.DataFrame(data['hourly'])
df['time'] = pd.to_datetime(df['time'])

In [5]:
df.head()

,time,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,rain,weather_code,cloud_cover,wind_direction_10m,wind_speed_10m,is_day
0,2024-01-01 00:00:00,20.9,82,21.4,0.0,0.0,3,100,127,15.7,0
1,2024-01-01 01:00:00,20.7,83,21.5,0.0,0.0,3,100,129,13.8,0
2,2024-01-01 02:00:00,20.5,83,21.5,0.0,0.0,3,100,133,11.7,0
3,2024-01-01 03:00:00,20.4,84,21.7,0.0,0.0,3,100,135,10.2,0
4,2024-01-01 04:00:00,20.3,86,21.3,2.4,1.9,63,100,131,12.5,0


In [6]:
# Salvar como CSV em memória
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

In [7]:
# Nome do arquivo com timestamp
now = datetime.now(timezone.utc).strftime('%Y-%m-%d_%H-%M-%S')
filename = f"dados_historicos/20240101-20241231.csv"

In [ ]:
# Enviar para o S3
try:
    s3 = boto3.client('s3')
    bucket_name = 'dados-clima'  
    s3.put_object(Bucket=bucket_name, Key=filename, Body=csv_buffer.getvalue())
    print('Dados carregados')
except Exception as e:
    print(f'Falha: {e}')